In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv('./training.csv').drop('EventId', 1)
test = pd.read_csv('./test.csv').drop('EventId', 1)

In [3]:
train.head(1)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,138.47,51.655,97.827,27.98,0.91,124.711,2.666,3.064,41.928,197.76,...,2,67.435,2.15,0.444,46.062,1.24,-2.475,113.497,0.002653,s


In [4]:
test.head(1)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,-999.0,79.589,23.916,3.036,-999.0,-999.0,-999.0,0.903,3.036,56.018,...,2.022,98.556,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-0.0


In [5]:
y_train, weights_train = train['Label'], train['Weight']
y_train = [1 if t == 's' else 0 for t in y_train]
train = train[test.columns]

print train.shape, test.shape

(250000, 30) (550000, 30)


### Заменим -999 (n/a) на средние значения и добавим соотв. фичи

In [6]:
cat_cols = []
def fillna(X):
    old_cols = X.columns
    for c in old_cols:
        if (len(X[X[c]==-999.]) > 0):
            # если n/a встречаются часто
            if (len(X[X[c]==-999.]) > 10):
                newname = c + '_is_na'
                cat_cols.append(newname)
                X[newname] = [1 if x==-999. else 0 for x in X[c]]
            mean = X[X[c]!=-999.][c].mean()
            X[c] = X[c].replace(-999., mean)

fillna(train)
fillna(test)

In [7]:
train.shape, test.shape

((250000, 41), (550000, 41))

## Посмотрим на распределения фичей

In [8]:
%%javascript
// IPython.OutputArea.auto_scroll_threshold = 9999;
IPython.OutputArea.auto_scroll_threshold = 200;
//(посмотрим на распределение всех фичей, потом сделать 200 !!!)

<IPython.core.display.Javascript object>

в ячейке ниже рисуется ~40 графиков, осторожно

In [9]:
# for c in train.columns:
#     if len(np.unique(train[c]))<100:
#         continue
#     print c
#     print 'nunuqie: ', len(np.unique(train[c])), '/', len(train[c])
#     print train[c].describe()
#     plt.figure(figsize=(10,5))
#     plt.hist(train[c], bins=200)
#     plt.show()
#     if len(train[train[c]<=0.]) > 40:
#         continue
#     print 'log of', c
#     plt.hist(np.log(train[train[c]>0][c]), bins=200)
#     plt.show()

### Многие фичи имеют логнормальное распределение:

DER_pt_ratio_lep_tau

DER_mass_vis 

PRI_met

PRI_met_sumet

### Добавим log вместо них, а также добавим log следующих колонок (заодно отнормируем еще раз)
DER_pt_tot

DER_sum_pt

### потому что они похожи на логарифм гамма-распределения



In [10]:
log_cols = ['DER_pt_ratio_lep_tau', 'DER_mass_vis', 'PRI_met', 'PRI_met_sumet']
for col in log_cols:
    train[col] = np.log(train[col]) - [np.mean(train[col])]*len(train)
    test[col] = np.log(test[col]) - [np.mean(test[col])]*len(test)

In [11]:
train['DER_pt_tot'].describe()

count    250000.000000
mean         18.917332
std          22.273494
min           0.000000
25%           2.841000
50%          12.315500
75%          27.591000
max        2834.999000
Name: DER_pt_tot, dtype: float64

In [12]:
gamma_log_cols = ['DER_pt_tot', 'DER_sum_pt']
for col in gamma_log_cols:
    # прибавим epsilon, там некоторые - нули
    train[col+'_log'] = np.log(train[col] + [.01]*len(train)) 
    test[col+'_log'] = np.log(test[col] + [.01]*len(test)) 

In [13]:
train.shape, test.shape

((250000, 43), (550000, 43))

### Отнормируем все фичи
кроме  
PRI_jet_num (это категориальный признак),

DER_pt_ratio_lep_tau,

DER_mass_vis,

PRI_met,

PRI_met_sumet,

DER_pt_tot,

DER_sum_pt,

In [14]:
cols = set(train.columns) - set(['PRI_jet_num', 
                               'DER_pt_ratio_lep_tau', 
                               'DER_mass_vis', 
                               'PRI_met', 
                               'PRI_met_sumet', 
                               'DER_pt_tot',
                               'DER_sum_pt']) 
cols = cols - set(cat_cols)

len(cols)

25

In [15]:
from scipy.stats import variation as var
for c in cols:
    train[c] = train[c]*(1./var(train[c])) - [train[c].mean()]*len(train)
    test[c] = test[c]*(1./var(test[c])) - [test[c].mean()]*len(test)

### Векторизуем PRI_jet_num (4 значения)

In [16]:
print len(np.unique(train['PRI_jet_num'])), len(np.unique(test['PRI_jet_num']))

4 4


In [17]:
train_dummy_enc = pd.get_dummies(train['PRI_jet_num'])
test_dummy_enc = pd.get_dummies(test['PRI_jet_num'])

colnames = ['PRI_jet_num_0', 'PRI_jet_num_1', 'PRI_jet_num_2', 'PRI_jet_num_3']
train_dummy_enc.columns = colnames
test_dummy_enc.columns = colnames

train = pd.concat([train, train_dummy_enc], axis=1)
test = pd.concat([test, test_dummy_enc], axis=1)

del train['PRI_jet_num'], test['PRI_jet_num']

In [18]:
train.shape, test.shape

((250000, 46), (550000, 46))

In [19]:
train.head(1)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_jet_leading_phi_is_na,PRI_jet_subleading_pt_is_na,PRI_jet_subleading_eta_is_na,PRI_jet_subleading_phi_is_na,DER_pt_tot_log,DER_sum_pt_log,PRI_jet_num_0,PRI_jet_num_1,PRI_jet_num_2,PRI_jet_num_3
0,198.024257,22.722161,-76.598781,-32.447609,-0.072969,-155.354537,-0.31288,6.914279,41.928,197.76,...,0,0,0,0,3.466975,37.324892,0.0,0.0,1.0,0.0


In [20]:
test.head()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_jet_leading_phi_is_na,PRI_jet_subleading_pt_is_na,PRI_jet_subleading_eta_is_na,PRI_jet_subleading_phi_is_na,DER_pt_tot_log,DER_sum_pt_log,PRI_jet_num_0,PRI_jet_num_1,PRI_jet_num_2,PRI_jet_num_3
0,162.262243,61.412103,-77.949356,-55.051020,3.75006,272.544429,1.189090,0.374748,3.036,56.018,...,1,1,1,1,-0.494135,27.194897,1.0,0.0,0.0,0.0
1,126.178716,44.587730,-76.647147,-12.148775,3.75006,272.544429,1.189090,3.860244,2.679,132.865,...,0,1,1,1,-0.682288,34.071867,0.0,1.0,0.0,0.0
2,152.747110,28.924474,-76.555834,-54.045113,3.75006,272.544429,1.189090,6.012424,4.137,97.600,...,1,1,1,1,-0.028424,31.615682,1.0,0.0,0.0,0.0
3,194.868132,-6.704427,-76.546228,-49.507112,3.75006,272.544429,1.189090,6.182894,9.104,94.112,...,1,1,1,1,1.160060,31.325904,1.0,0.0,0.0,0.0
4,51.017393,65.838252,-77.050936,24.078488,1.04074,556.868147,0.977312,0.755261,77.213,721.552,...,0,0,0,0,4.385317,47.546246,0.0,0.0,0.0,1.0


# Training

In [21]:
from scipy.stats import pearsonr
pearsonr(weights_train, y_train)

(-0.63098171597333352, 0.0)

## Сначала для теста предскажем Weight, обучившись на Weight из X, после уже будем предсказывать Label. Почему?
### 1) получится своеобразный стекинг 2 алгоритмов, что уже хорошо, так как веса и лейбелы слабо коррелируют
### 2) будем использовать информацию о весах

'

In [30]:
params = {} 
params['objective']           = "reg:linear"
params['eval_metric']         = "rmse"
params['eta']                 = 0.01
params['subsample']           = 0.681
params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1

In [25]:
old_cols = train.columns

In [26]:
train.columns = [str(i) for i in range(0, len(train.columns))]
test.columns = [str(i) for i in range(0, len(test.columns))]

In [27]:
from sklearn.cross_validation import train_test_split as tts

(train_cv, 
 test_cv, 
 y_train_cv, 
 y_test_cv) = tts( train, y_train, 
                   test_size=0.3, 
                   random_state=42)
#                    stratify=y_train )

In [31]:
import xgboost as xgb

dtrain = xgb.DMatrix(train_cv, label=y_train_cv)
dval = xgb.DMatrix(test_cv, label=y_test_cv)
watchlist = [(dval,'eval')]

num_trees = 888
esr = 10

gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, 
                verbose_eval=False, early_stopping_rounds = esr)

Will train until eval error hasn't decreased in 10 rounds.


In [ ]:
# [154]	eval-rmse:0.338095

In [ ]:
param_grid = {'max_depth': range(5, 7),
              'objective':['reg:linear'],
              'n_estimators': range(100, 500),
              'learning_rate': np.arange(0.01, 0.3, .001), #eta
              'subsample': np.arange(0.5, 1, .01),
              'colsample_bytree': np.arange(0.5, 1, .01),
             }

In [ ]:
# train_columns = train.columns
train.columns = [str(i) for i in range(0, len(train_columns))]

In [ ]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

xgb_model = xgb.XGBRegressor()
n_iter_search=1
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_grid,
                                   n_iter=n_iter_search, scoring="mean_squared_error",
                                   n_jobs=8, verbose=10)

random_search.fit(train, weights_train) 

In [ ]:
xgb.XGBRegressor().get_params().keys()

In [ ]:
print random_search.grid_scores_
print random_search.best_estimator_
print random_search.best_score_
print random_search.best_params_


In [ ]:
# import xgboost as xgb

# gbm = xgb.XGBRegressor()
# gbm_params = {
#     'learning_rate': [0.005, 0.2],
#     'n_estimators': [100, 1000],
#     'max_depth': [2, 3, 10],
# }
# cv = StratifiedKFold(train_y)
# grid = GridSearchCV(gbm, gbm_params,scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)
# grid.fit(train_X, train_y)

# print (grid.best_params_)